### Import Libraries

In [364]:
import pandas as pd
import pickle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize   
from nltk.stem import WordNetLemmatizer 

<br>

### Create Lemmatizer Class to use in CountVectorizer
##### I got the code and understanding of how to use it from here https://stackoverflow.com/questions/47423854/sklearn-adding-lemmatizer-to-countvectorizer

In [358]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]        

<br>

### Open and Clean Data Sources

In [359]:
# Open and check data source
with open('Leslie_all.pickle','rb') as read_file:
    leslie_episodes = pickle.load(read_file)

leslie_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Leslie Knope,s1e01,s1e01,"Hello. Hi. My name is Leslie Knope, and I work..."
1,Leslie Knope,s1e02,s1e02,"Well, one of the funner things that we do here..."
2,Leslie Knope,s1e03,s1e03,The Parks Department has so many programs. Jer...
3,Leslie Knope,s1e04,s1e04,"I don't believe it. Oh, my God. It's real. Hey..."
4,Leslie Knope,s1e05,s1e05,"In a town as old as Pawnee, there's a lot of h..."


In [360]:
# Remove digits, any words less than 3 characters, and punctuation
leslie_episodes['Episode_Text'] = leslie_episodes['Episode_Text'].str.replace('\d+', '') # for digits
leslie_episodes['Episode_Text'] = leslie_episodes['Episode_Text'].str.replace(r'(\b\w{1,2}\b)', '') # for words
leslie_episodes['Episode_Text'] = leslie_episodes['Episode_Text'].str.replace('[^\w\s]', '') # for punctuation 
leslie_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Leslie Knope,s1e01,s1e01,Hello name Leslie Knope and work for the P...
1,Leslie Knope,s1e02,s1e02,Well one the funner things that here Pawne...
2,Leslie Knope,s1e03,s1e03,The Parks Department has many programs Jerry ...
3,Leslie Knope,s1e04,s1e04,don believe God real Hey Hey Hello Boys ...
4,Leslie Knope,s1e05,s1e05,town old Pawnee there lot history every...


In [361]:
# Open and check data source
with open('Ron_all.pickle','rb') as read_file:
    ron_episodes = pickle.load(read_file)

ron_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Ron Swanson,s1e01,s1e01,Tonight is our next monthly community outreach...
1,Ron Swanson,s1e02,s1e02,"Uh, sure, Paul. What can I do for you? Yeah, a..."
2,Ron Swanson,s1e03,s1e03,"No comment. Hey, Haverford, maybe one day you'..."
3,Ron Swanson,s1e04,s1e04,Go to jail? What's going on? Put it in an emai...
4,Ron Swanson,s1e05,s1e05,The only reason anybody's going to this thing ...


In [362]:
# Remove digits, any words less than 3 characters, and punctuation
ron_episodes['Episode_Text'] = ron_episodes['Episode_Text'].str.replace('\d+', '') # for digits
ron_episodes['Episode_Text'] = ron_episodes['Episode_Text'].str.replace(r'(\b\w{1,2}\b)', '') # for words
ron_episodes['Episode_Text'] = ron_episodes['Episode_Text'].str.replace('[^\w\s]', '') # for punctuation 
ron_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Ron Swanson,s1e01,s1e01,Tonight our next monthly community outreach p...
1,Ron Swanson,s1e02,s1e02,sure Paul What can for you Yeah absolutely ...
2,Ron Swanson,s1e03,s1e03,comment Hey Haverford maybe one day you figur...
3,Ron Swanson,s1e04,s1e04,jail What going Put email Let not blow t...
4,Ron Swanson,s1e05,s1e05,The only reason anybody going this thing bec...


<br>

### Final Corex Topic Models

#### Leslie Final Corex Topic Model

In [363]:
# Manually created stop words
commonwords = ['use', 'own', 'barely', 'bottom']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 

stop_words = list(set(commonwords + names))

In [365]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, 
                             tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(leslie_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

In [366]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=leslie_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,department,director,recreation,live,choice,summer,further,subcommittee,neighborhood
1: government,federal,wheel,turning,intern,ready,brainstorming,themselves,moved,truck


<br>

#### Ron Final Corex Topic Model

In [367]:
# Manually created stop words and nltk stopwords
commonwords = ['become', 'new', 'even']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 

nltk_stopwords =  stopwords.words('english')

stop_words = list(set(names + commonwords + nltk_stopwords))

In [368]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, 
                             tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(ron_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

/Users/madelinevossbrinck/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [369]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=ron_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,drive,want,job,killed,nothing,girl,better,person,would
1: government,office,taxpayer,meet,large,crap,stand,dollar,hired,wasting


<br>

### Additional CorEx Topic Models
##### This is not a comprehensive list of all that were tried, just some of the alternatives

#### Additional Models for Leslie

In [417]:
# Manually created stop words and nltk stopwords
commonwords = ['wheel', 'turning','use', 'barely', 'bottom', 'except']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 

nltk_stopwords =  stopwords.words('english')

stop_words = list(set(nltk_stopwords + names + commonwords))

In [418]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(leslie_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

In [419]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=leslie_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,department,director,live,pit,year,candidate,system,subcommittee,closed
1: government,summer,clear,intern,recreation,prepared,community,someday,deputy,cake


In [429]:
# Manually created stop words and nltk stopwords
commonwords = ['use', 'extra']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 


nltk_stopwords =  stopwords.words('english')

stop_words = list(set(names + nltk_stopwords + commonwords))

In [430]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(leslie_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

In [431]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=leslie_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,director,department,recreation,choice,water,councilwoman,year,neighborhood,sell
1: government,wheel,wave,intern,cake,bottom,barely,open,zero,notice


<br>

#### Additional Models for Ron

In [398]:
# Manually created stop words and nltk stopwords
commonwords = ['cream', 'cheese', 'want', 'new', 'large', 'made', 'small', 'throat', 'old', 'feel' ,'must']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 

nltk_stopwords =  stopwords.words('english')

stop_words = list(set(names + nltk_stopwords + commonwords))

In [399]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(ron_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

In [400]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=ron_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,job,complete,need,department,run,six,drive,person,help
1: government,taxpayer,stand,offering,budget,buddy,wasting,tax,show,win


In [391]:
# Manually created stop words and nltk stopwords
commonwords = ['new', 'expect', 'tomorrow', 'even', 'must', 'told']

names = ['leslie', 'knope', 'ron', 'swanson', 'ben', 'wyatt', 'april', 'ludgate', 'tom', 'haverford', 
         'ann', 'perkins', 'andy', 'dwyer', 'jerry', 'gergich', 'donna', 'meagle', 'tommy', 'mark', 'brendanawicz'] 

nltk_stopwords =  stopwords.words('english')

stop_words = list(set(commonwords + names + nltk_stopwords))

In [392]:
# Transform data using CountVectorizer
vectorizer = CountVectorizer(max_features=20000, 
                             stop_words=stop_words, tokenizer = LemmaTokenizer(),
                             binary=True)

doc_word = vectorizer.fit_transform(ron_episodes.Episode_Text)
words = list(np.asarray(vectorizer.get_feature_names()))

In [393]:
# Run topic model
topic_model = ct.Corex(n_hidden=2, words=words,
                       max_iter=200, verbose = False, seed = 1)

topic_model.fit(doc_word, words=words, docs=ron_episodes.Episode_Text, 
                anchors=[['park'],
                         ['government']]
                         , anchor_strength=2)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: park,want,speak,station,federal,fit,definitely,throw,complicated,invite
1: government,crap,office,taxpayer,made,right,city,think,buddy,tax
